In [1]:
import os

import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from kalman_vae import KalmanVariationalAutoencoder
from bouncing_ball.dataloaders.bouncing_data import BouncingBallDataLoader

In [2]:
dataloader_train = BouncingBallDataLoader(root_dir='bouncing_ball/datasets/bouncing-ball/train')
dataloader_test = BouncingBallDataLoader(root_dir='bouncing_ball/datasets/bouncing-ball/test')

In [3]:
dataloader_train = DataLoader(dataloader_train, batch_size=64, shuffle=True)
dataloader_test = DataLoader(dataloader_test, batch_size=64, shuffle=True)

In [4]:
for i, data in enumerate(dataloader_train):
    print(data.shape)
    # To Float32
    data = (data > 0.5).float()
    break

torch.Size([64, 50, 1, 16, 16])


In [5]:
kvae = KalmanVariationalAutoencoder(image_size = data.shape[3:], image_channels=data.shape[2], a_dim=2, z_dim=4, K=3, decoder_type='bernoulli')

In [6]:
optimizer = torch.optim.Adam(kvae.parameters(), lr=1e-3)

In [7]:
os.makedirs('checkpoints/bouncing_ball', exist_ok=True)
losses = []
p = tqdm(range(10))
for epoch in p:
    for i, data in enumerate(dataloader_train):
        data = (data > 0.5).float()
        optimizer.zero_grad()
        elbo, info = kvae.elbo(data)
        loss = -elbo
        loss.backward()
        optimizer.step()
        p.set_description("Epoch: %d, Loss: %.4f" % (epoch, loss.item()))
    losses.append(loss.item())

    torch.save(kvae.state_dict(), 'checkpoints/bouncing_ball/kvae-%d.pth' % epoch)

  0%|                                                    | 0/10 [00:00<?, ?it/s]

> /Users/naoki/Documents/PhD/kalman-vae/ssm.py(205)state_transition_log_likelihood()
    203         import pdb; pdb.set_trace()
    204 
--> 205         state_transition_log_likelihood = 0.0
    206 
    207         for t in range(sequence_length):

    192     def state_transition_log_likelihood(self, zs, mat_As):
    193         sequence_length, batch_size, _, _ = zs.size()
    194 
    195         # Initial state estimate: \hat{z}_{0|-1}
    196         mean_t_plus = (
    197             self.initial_state_mean.unsqueeze(0).repeat(batch_size, 1).unsqueeze(2)
    198         )
    199 
    200         # Initial state covariance: \Sigma_{0|-1}
    201         cov_t_plus = self.initial_state_covariance.unsqueeze(0).repeat(batch_size, 1, 1)
    202 
    203         import pdb; pdb.set_trace()
    204 
--> 205         state_transition_log_likelihood = 0.0
    206 
    207         for t in range(sequence_length):
    208             if not torch.isnan(zs[t]).any():
    209              

  0%|                                                    | 0/10 [00:37<?, ?it/s]
